# **학업 일지**

#### 오늘의 한마디  

> **경사하강법에 대한 기본이해를 바탕으로 코드를 작성할 수 있어야한다.**

## 1. 강의 정리

#### *Numpy 연습을 통한 벡터와 행렬의 기본개념 숙지*

### 1.1 Gradient descent

#### 1.1.1 Sympy

- 미분 식 계산

In [2]:
import sympy as sym
from sympy.abc import x
print(sym.diff(sym.poly(x**2 + 2*x + 3),x))

Poly(2*x + 2, x, domain='ZZ')


#### 1.1.2 Gradient Descent(GD) 구현

- 데이터가 목표지점에 도달하기 위해서 회귀 계수를 갱신한다.

In [3]:
import sympy as sym
import numpy as np
from sympy.abc import x

def func(val):
    """
    Initialize function
    """
    fun = sym.poly(x**2 + 2*x + 3)
    return fun.subs(x, val), fun

def func_gradient(fun, val):
    """
    Find a value of gradient
    """
    _, function = fun(val)
    diff = sym.diff(function, x)
    return diff.subs(x, val), diff

def gradient_descent(fun, init_point, lr_rate=1e-2, epsilon = 1e-5):
    """
    Find minimum value of function by using GD
    """
    cnt = 0
    val = init_point
    diff, _ = func_gradient(fun, init_point)
    while np.abs(diff) > epsilon:
        val = val - lr_rate*diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    print(f'function: {fun(val)}, cnt = {cnt}, min = {fun(val)[0]}')

if __name__ == '__main__':
    gradient_descent(fun=func, init_point=np.random.uniform(-2,2))

function: (2.00000000002418, Poly(x**2 + 2*x + 3, x, domain='ZZ')), cnt = 618, min = 2.00000000002418


선형대수(일차식의 집합연산)같은 경우에는 Pseudo-inverse matrix의 선형회귀를 통해 최적화 할 수 있지만 일반적인 식인 경우에는 구하기 어렵기 때문에 SGD를 이용한다.

#### 1.1.3 GD with partial defferentiation

- 데이터가 목표지점에 가까워지도록 선형

In [4]:
# 무엇이 정답과 다른지 분석해보자
import sympy as sym
import numpy as np
from sympy.abc import x, y

def eval(fun, val):
    val_x, val_y = val
    fun_eval = fun.subs(x, val_x).subs(y, val_y)
    return fun_eval
    
def func(val):
    x_val, y_val = val
    fun = sym.poly(x**2 + 2*y**2 + 5*x*y)
    return eval(fun, val), fun

def gradient(fun, val):
    x_val, y_val = val
    _, fun = fun(val)
    diff_x = sym.diff(fun, x)
    diff_y = sym.diff(fun, y)
    grad_vec = np.array([eval(diff_x, [x_val,y_val]),eval(diff_y, [x_val, y_val])],dtype=float)
    return grad_vec

def gradient_descent(fun, init, lr=1e-2, epsilon=1e-5):
    cnt = 0
    val = init
    _, fun = fun(val)
    diff, _ = gradient(fun, val)
    while np.linalg.norm(diff) > epsilon:
        val = val - lr * diff
        diff = gradient(fun, init_val)
        cnt += 1
                                       


In [5]:
import numpy as np
import sympy as sym
from sympy.abc import x, y

def eval_(fun, val):
    val_x, val_y = val
    fun_eval = fun.subs(x, val_x).subs(y, val_y)
    return fun_eval

def func_multi(val):
    x_, y_ = val
    func = sym.poly(x**2 + 2*y**2)
    return eval_(func, [x_,y_]), func

def func_gradient(fun, val):
    x_, y_ = val
    _, function = fun(val)
    diff_x = sym.diff(function, x)
    diff_y = sym.diff(function, y)
    grad_vec = np.array([eval_(diff_x, [x_,y_]), eval_(diff_y, [x_, y_])], dtype=float)
    return grad_vec, [diff_x, diff_y]

def gradient_descent(fun, init_point, lr=1e-2, epsilon=1e-4):
    cnt=0
    val =init_point
    diff, _ = func_gradient(fun,val)
    while (np.linalg.norm(diff) > epsilon):
        val = val - lr*diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    
    print(f'function: {fun(val)[1]}, cnt: {cnt}, min: {fun(val)[0]}')

gradient_descent(fun=func_multi,init_point=[np.random.uniform(-2,2), np.random.uniform(-2,2)])
                  

function: Poly(x**2 + 2*y**2, x, y, domain='ZZ'), cnt: 460, min: 2.49952603754474E-9


#### 1.1.4 벡터에서의 경사하강법을 이용한 선형회귀 알고리즘
- 역행렬을 구하지 않고 회귀계수를 구할 수 있다.

In [7]:
x = np.array([[1,1],[1,2],[2,2],[2,3]])
y = np.array(np.dot(x, np.array([1,2])))

beta_gd = [10.1, 15.1, -6.5]
x_ = np.array([np.append(i,[-1]) for i in x])
print(x_)
for t in range(5000):
    error = y - x_ @ beta_gd
    grad = - np.transpose(x_) @ error
    beta_gd = beta_gd - 0.01 * grad
    if t % 1000 == 0:
        print(error, grad, beta_gd)

print(y)
print(x_)
print(beta_gd)
print(x_ @ beta_gd)

[[ 1  1 -1]
 [ 1  2 -1]
 [ 2  2 -1]
 [ 2  3 -1]]
[-28.7 -41.8 -50.9 -64. ] [ 300.3  406.1 -185.4] [ 7.097 11.039 -4.646]
[-0.01134889 -0.01550333  0.01070553  0.0065511 ] [-0.00766103  0.0012912  -0.0095956 ] [ 0.97386775  2.00414152 -0.03330736]
[-0.00064241 -0.00084632  0.00057735  0.00037343] [-4.12832426e-04  6.00600768e-05 -5.37948298e-04] [ 9.98580455e-01  2.00020332e+00 -1.85678072e-03]
[-3.56327344e-05 -4.65812990e-05  3.16921433e-05  2.07435787e-05] [-2.26574107e-05  3.18030954e-06 -2.97783114e-05] [ 9.99921953e-01  2.00001092e+00 -1.02659829e-04]
[-1.96799189e-06 -2.56839222e-06  1.74642342e-06  1.14602310e-06] [-1.24850892e-06  1.73860195e-07 -1.64393759e-06] [ 9.99995698e-01  2.00000060e+00 -5.66596783e-06]
[3 5 6 8]
[[ 1  1 -1]
 [ 1  2 -1]
 [ 2  2 -1]
 [ 2  3 -1]]
[ 9.99999762e-01  2.00000003e+00 -3.13502468e-07]
[3.00000011 5.00000014 5.9999999  7.99999994]


#### 1.1.5 Mini batch GD 직접 구현

![sgd.png](./img/sgd.png)

In [86]:
# 데이터 확인
x = np.array([[1,1],[1,2],[2,2],[2,3]]) # (4,2)
y = np.array(np.dot(x, np.array([1,2]).T)) + 3 # (4,)
beta_gd = np.array([10.1, 15.1, -6.5])[np.newaxis,:].T # (3,1)
y = y[:,np.newaxis] # (4,1)
lr = 0.01
eps = 0.005
x_ = np.array([np.append(i,[1]) for i in x]) # (4,3)
mini_x = []
mini_y = []
batch = 2
for i in range(len(x_)//batch):
    mini_x.append(x_[i*batch:(i+1)*batch])
    mini_y.append(y[i*batch:(i+1)*batch])
for i in range(10000):
    divide_standard = 1
#     print(mini_x, mini_y)
    if i%2 == 0:
        divide_standard = 0
    test_x = mini_x[divide_standard] # (2,3)
    test_y = mini_y[divide_standard] # (2,1)
#     print('test',test_x,test_y, beta_gd, test_x.shape)
    y_ = test_x.dot(beta_gd)
#     print('??',y_)
    error = test_y - y_ # (2,1)
#     print('error',error,error.shape)
    beta_gd = beta_gd + 2/batch*lr*(test_x.T).dot(error)

print(beta_gd)
# label이 결정되면 그 모양을 만드는 batch만 만들수있네 이 경우에는 입력이 한 개 
# 출력이 한개

[[1.00000145]
 [1.99999976]
 [2.9999981 ]]


In [90]:
x_@beta_gd, y

(array([[ 5.99999931],
        [ 7.99999907],
        [ 9.00000053],
        [11.00000028]]),
 array([[ 6],
        [ 8],
        [ 9],
        [11]]))

수렴에 더 오래걸리지만 답에 근사하는 것을 볼 수 있었다.

## 2. 피어 세션 정리

#### 2.1 스터디

- 2.1.1 Numpy의 기본 활용 정리와 실제로 쓰일 수 있는 함수에 대해 리뷰하는 시간을 가졌다.(발표: 변재경)
np.newaxis의 활용

- 2.1.2 행렬에 대한 깊은 이해와 수학적 표현에 대해 토론하는 시간을 가졌다.(발표: 김상현)


#### 2.2 원하는 진로를 잡고 논문 공부

- Deep Learning : Yann LeCnn, Yoshua Bengio & Geoffrey Hinton
딥러닝에 대한 전반적인 이해를 돕고 AI 논문에 대한 언어 장벽을 없애기 위해 필요하며 발표 예정

## 3. 진행중인 공부 및 신규 공부 목록

- 진행중인 공부  

    - Deep learning 논문 읽고 정리하기
    - AI 기본 수학 : Mathematics for Machine learning - Marc Peter Deisenroth
    - 파이썬 파일관리를 기본으로 해서 프로젝트 하나 진행해보기
    - 웹 크롤링 및 데이터 처리 연습 익숙해지기

- 신규 공부 목록  
 
 - Numpy를 이용하여 프로젝트 하나 진행해보기 (파일 관리를 기본으로 한 프로젝트와 통합)

- 완료한 공부  

## 4. 감사한 일

- 항상 질문을 많이하는데 친절하게 대답해주는 우리 팀에게 감사합니다.